In [77]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import itertools

%matplotlib inline

In [78]:
dataset = pd.read_csv('data/boston-housing.csv')

In [79]:
dataset.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [80]:
train_data, test_data = train_test_split(dataset, test_size=0.2)

In [81]:
#standard_scalar = StandardScaler()

In [82]:
def preprocess(standard_scalar, data_features):
    data_features = pd.DataFrame(standard_scalar.fit_transform(data_features), columns=data_features.columns)
    return data_features

In [83]:
train_target = train_data['medv']
del train_data['medv']
train_features = train_data

test_target = test_data['medv']
del test_data['medv']
test_features = test_data

In [84]:
train_features = preprocess(standard_scalar, train_features)
test_features = preprocess(standard_scalar, test_features)

In [85]:
COLUMNS = train_features.columns
TARGET = "medv"
print(COLUMNS)

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'black', 'lstat'],
      dtype='object')


In [86]:
feature_values = [ tf.contrib.layers.real_valued_column(feature)
    for feature in COLUMNS]

In [87]:
def input_fn(features, target):
    features_data = {feature:tf.constant(features[feature].values) for feature in features}
    labels_data = tf.constant(target.values)
    return features_data, labels_data

In [96]:
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_values, optimizer='RMSProp', 
                                          hidden_units=[10, 10],
                                          model_dir='D:/Git/Coding2Fun/Blog/DeepLearning/Part-I/boston/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_evaluation_master': '', '_task_id': 0, '_is_chief': True, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A88A0056A0>, '_num_worker_replicas': 0, '_master': '', '_environment': 'local', '_model_dir': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_keep_checkpoint_every_n_hours': 10000}


In [106]:
features_data, target_data = input_fn(test_features, test_target) 
early_stop = validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    features_data,
    target_data,
    every_n_steps=500,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=50)

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


In [107]:
regressor.fit(input_fn = lambda: input_fn(train_features, train_target), steps=5000, monitors=[early_stop])

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from D:/Git/Coding2Fun/Blog/DeepLearning/Part-I/boston/model.ckpt-5001
INFO:tensorflow:Saving checkpoints for 5002 into D:/Git/Coding2Fun/Blog/DeepLearning/Part-I/boston/model.ckpt.
INFO:tensorflow:loss = 3.08712, step = 5002
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Sciki

ValueError: Inputs cannot be tensors. Please provide input_fn.

In [99]:
ev = regressor.evaluate(input_fn = lambda :input_fn(test_features, test_target), steps=10)["accuracy"]
ev

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-04-04:55:35
INFO:tensorflow:Restoring parameters from D:/Git/Coding2Fun/Blog/DeepLearning/Part-I/boston/model.ckpt-5000
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2017-05-04-04:55:35
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 19.5685


KeyError: 'accuracy'